In [1]:

!conda install --yes -c conda-forge pdal python-pdal gdal 
!conda create --yes -n pdalpy python=3.8
!conda activate --yes pdalpy
!pip install dask distributed laserfarm
!conda upgrade --yes numpy
!conda update --yes -n base conda


Solving environment: done

## Package Plan ##

  environment location: /home/alogo/anaconda3/envs/jupyterlab

  added / updated specs:
    - gdal
    - pdal
    - python-pdal


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2021.10.26~ --> conda-forge::ca-certificates-2021.10.8-ha878542_0
  certifi            pkgs/main::certifi-2021.10.8-py39h06a~ --> conda-forge::certifi-2021.10.8-py39hf3d152e_1
  openssl              pkgs/main::openssl-1.1.1m-h7f8727e_0 --> conda-forge::openssl-1.1.1l-h7f98852_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  environment location: /home/alogo/anaconda3/envs/pdalpy

  added / updated specs:
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    sqlite-3.37.2              |       hc21

In [2]:
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

## Global Configuration

In [3]:
# Configurations.sp
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

conf_remote_path_root = pathlib.Path('/webdav')
conf_remote_path_ahn = pathlib.Path('/webdav/ahn')
conf_remote_path_retiled = pathlib.Path('/webdav/retiled/')
conf_remote_path_targets = pathlib.Path('/webdav/targets')
conf_local_tmp = pathlib.Path('/tmp')

webdav_hostname = 'https://lifewatch.lab.uvalight.net:32443'
webdav_login = 'ZDFmNDg0Nzk0ZTJi'
webdav_password = 'NDBjMmMzODJkYTBh'

conf_wd_opts = {
    'webdav_hostname': webdav_hostname,
    'webdav_login': webdav_login,
    'webdav_password': webdav_password
}

## Fetching Laz Files from remote WebDAV

In [5]:
# Fetch Laz Files.sp

laz_files = [f for f in list_remote(get_wdclient(conf_wd_opts), conf_remote_path_ahn.as_posix())if f.lower().endswith('.laz')]
print(laz_files)

['C_19HZ2.LAZ', 'C_01GN2.LAZ']


## Retiling of big files into smaller tiles

In [7]:
# Retiling.sp

remote_path_retiled = conf_remote_path_retiled

grid_retile = {
    'min_x': -113107.81,
    'max_x': 398892.19,
    'min_y': 214783.87,
    'max_y': 726783.87,
    'n_tiles_side': 512
}


retiling_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_ahn.as_posix(),
    'set_grid': grid_retile,
    'split_and_redistribute': {},
    'validate': {},
    'pushremote': conf_remote_path_retiled.as_posix(),
    'cleanlocalfs': {}
}

file = laz_files[0]
# To run parallel after spliter 
# file = laz_files
retiler = Retiler(file).config(retiling_input).setup_webdav_client(conf_wd_opts)
retiler.run()

2022-01-03 14:50:28,410 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/pipeline_input
2022-01-03 14:50:28,411 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/pipeline_output
2022-01-03 14:50:28,412 -           laserfarm.pipeline_remote_data -       INFO - Pulling from WebDAV /webdav/ahn/C_19HZ2.LAZ ...
2022-01-03 14:50:29,820 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2022-01-03 14:50:29,822 -                        laserfarm.retiler -       INFO - Setting up the target grid
2022-01-03 14:50:29,826 -                        laserfarm.retiler -       INFO - Splitting file /tmp/pipeline_input/C_19HZ2.LAZ with PDAL ...
2022-01-03 14:50:30,053 -                        laserfarm.retiler -       INFO - ... splitting completed.
2022-01-03 14:50:30,055 -                        laserfarm.retiler -       INFO - Redistributing files to tiles ...
2022-01-03 14:50:30,058 -                        l

## Fetching retilied files (tiles) from remote WebDAV

In [8]:
# Fetch Tiles.sp

tiles = [t.strip('/') for t in list_remote(get_wdclient(conf_wd_opts), remote_path_retiled.as_posix())
         if fnmatch.fnmatch(t, 'tile_*_*/')]

## Extract features from tiles

In [9]:
# Feature Extraction.sp

t = tiles[0]

tile_mesh_size = 10.
features = ['perc_95_normalized_height']

grid_feature = {
    'min_x': -113107.81,
    'max_x': 398892.19,
    'min_y': 214783.87,
    'max_y': 726783.87,
    'n_tiles_side': 512
}

feature_extraction_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_retiled.as_posix(),
    'load': {'attributes': ['raw_classification']},
    'normalize': 1,
    'apply_filter': {
        'filter_type': 'select_equal', 
        'attribute': 'raw_classification',
        'value': [1, 6]#ground surface (2), water (9), buildings (6), artificial objects (26), vegetation (?), and unclassified (1)
    },
    'generate_targets': {
        'tile_mesh_size' : tile_mesh_size,
        'validate' : True,
        **grid_feature
    },
    'extract_features': {
        'feature_names': features,
        'volume_type': 'cell',
        'volume_size': tile_mesh_size
    },
    'export_targets': {
        'attributes': features,
        'multi_band_files': False
    },
    'pushremote': conf_remote_path_targets.as_posix(),
#     'cleanlocalfs': {}
}


idx = (t.split('_')[1:])
processing = DataProcessing(t, tile_index=idx).config(feature_extraction_input).setup_webdav_client(conf_wd_opts)
processing.run()

2022-01-03 14:50:45,119 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/pipeline_input
2022-01-03 14:50:45,121 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/pipeline_output
2022-01-03 14:50:45,124 -           laserfarm.pipeline_remote_data -       INFO - Pulling from WebDAV /webdav/retiled/tile_278_391 ...
2022-01-03 14:50:47,763 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2022-01-03 14:50:47,765 -                laserfarm.data_processing -       INFO - Loading point cloud data ...
2022-01-03 14:50:47,765 -                laserfarm.data_processing -       INFO - ... loading /tmp/pipeline_input/tile_278_391/C_01GN2_1.LAZ
2022-01-03 14:50:47,802 -                          pylas.lasreader -      ERROR - lazrs failed to decompress points: lazrs is not installed
2022-01-03 14:51:01,229 -                laserfarm.data_processing -       INFO - ... loading completed.
2022-01-03 14:51:01,231

In [10]:
# GeoTIFF Export

feature = features[0]
remote_path_geotiffs = conf_remote_path_ahn.parent / 'geotiffs'

# setup input dictionary to configure the GeoTIFF export pipeline
geotiff_export_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_targets.as_posix(),
    'parse_point_cloud': {},
    'data_split': {'xSub': 1, 'ySub': 1},
    'create_subregion_geotiffs': {'output_handle': 'geotiff'},
    'pushremote': remote_path_geotiffs.as_posix(),
    'cleanlocalfs': {}   
}

writer = GeotiffWriter(input_dir=feature, bands=feature).config(geotiff_export_input).setup_webdav_client(conf_wd_opts)
writer.run()

2022-01-03 15:09:07,343 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/pipeline_input
2022-01-03 15:09:07,344 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/pipeline_output
2022-01-03 15:09:07,347 -           laserfarm.pipeline_remote_data -       INFO - Pulling from WebDAV /webdav/targets/perc_95_normalized_height ...
2022-01-03 15:09:09,343 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2022-01-03 15:09:09,345 -                 laserfarm.geotiff_writer -       INFO - 1 PLY files found
2022-01-03 15:09:09,452 -                 laserfarm.geotiff_writer -       INFO - No. of points per file: 10000
2022-01-03 15:09:09,454 -                 laserfarm.geotiff_writer -       INFO - Resolution: (10.0m x 10.0m)
2022-01-03 15:09:09,455 -                 laserfarm.geotiff_writer -       INFO - Splitting data into (1x1) sub-regions
2022-01-03 15:09:09,456 -                 laserfarm.geotiff_write